# Evaluating the Spread of AI-Generated Synthetic Media on X - Main



## 1. Prepare Environment and Load Data

In [ ]:
from google.colab import drive
import pandas as pd
from datetime import datetime
drive.mount('/content/drive')

DATA_LOC = "path-to-notes-tsv-file"
communitynotes_data = pd.read_csv(DATA_LOC, delimiter = '\t')

communitynotes_data["date"] = pd.to_datetime(communitynotes_data['createdAtMillis'], unit='ms')
communitynotes_data["date"] = communitynotes_data["date"].dt.strftime('%d/%m/%Y')
communitynotes_data = communitynotes_data[(pd.to_datetime(communitynotes_data['date'], format='%d/%m/%Y') > '01-11-2022') & (pd.to_datetime(communitynotes_data['date'], format='%d/%m/%Y') < '30-09-2023')]

## 2. Extract Community Notes by Keyword and Save Data as Dictionary

This query filters for instances of AI-generated visual content that could be misleading, such as "deepfakes" or AI-generated images. However, it deliberately excludes strings related to explicitly AI-generated content, such as the hashtag #aiart, focusing on capturing only those cases where the audience could potentially be misled due to the absence of clear labeling.

In [ ]:
import pandas as pd
import pickle

def filter_dataframe(full_df, expressions):
    pattern = '|'.join(expressions)
    new_df = full_df[full_df['summary'].str.contains(pattern, case=False, na=False)]
    return new_df

expressions = ["ai-generated(?:\s+image|\s+video|\s+art|\s+photo|\s+deepfake)",
               "ai generated(?:\s+image|\s+video|\s+art|\s+photo|\s+deepfake)",
               "ai(?:\s+image|\s+video|\s+art|\s+photo|\s+deepfake)",
               "ai-(?:\s+image|\s+video|\s+art|\s+photo|\s+deepfake)",
               "generated\s+(?:with|by)\s+(?:ai|artificial intelligence)",
               "midjourney", "stable diffusion", "dall-e", "deepfake",
               "deep fake", "deepfaked"]

visual_disinformation = filter_dataframe(communitynotes_data, expressions)
no_visual_disinformation = communitynotes_data.drop(visual_disinformation.index)

community_notes = {'visual': visual_disinformation, 'no_visual': no_visual_disinformation}

with open('community-notes-filtered.pkl', 'wb') as f:
    pickle.dump(community_notes, f)

## 3. Collect Data From Source Tweets

Here leverage Selenium's headless browser capabilities to systematically obtain data from the tweets to which the filtered community notes refer. The scraped data encompasses a wide range of metrics, including usernames, follower counts, tweet impressions, retweets, likes, bookmarks, as well as the tweet content itself.  Additionally, this pipeline is designed to automatically extract and store images embedded within these tweets, saving them to a Google Drive folder for subsequent analysis. It should be noted that, due to Twitter's dynamic content loading, the current implementation is unable to capture video content.

### 3.1 Prepare Environment and Create Chrome Driver for Selenium

In [ ]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import re
import pandas as pd
import urllib.request
from google.colab import drive

def create_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome(options=chrome_options)

### 3.2 Run the Main Data Collection Pipeline

In [ ]:
def generate_tweet_url(tweet_id):
    return f"https://twitter.com/anyuser/status/{tweet_id}"

def collect_tweet_data(source_tweets,OUTPUT_DIR_IMAGES, OUTPUT_FILE_INTERMEDIATE):
    drive.mount('/content/drive')
    driver = create_chrome_driver()
    source_tweets['tweetUrl'] = source_tweets['tweetId'].apply(generate_tweet_url)

    patterns = {
        'views': re.compile(r'([\d,.]+[KkMm]?)\s*\n\s*Views'),
        'reposts': re.compile(r'([\d,.]+[KkMm]?)\s*\n\s*Reposts'),
        'quotes': re.compile(r'([\d,.]+[KkMm]?)\s*\n\s*Quotes'),
        'likes': re.compile(r'([\d,.]+[KkMm]?)\s*\n\s*Likes'),
        'bookmarks': re.compile(r'([\d,.]+[KkMm]?)\s*\n\s*Bookmarks')
    }

    df = pd.DataFrame(columns=['tweetId', 'tweetDate', 'username', 'count', 'tweetUrl', 'imageUrl', 'views', 'reposts', 'quotes', 'likes', 'bookmarks', 'text'])
    os.makedirs(OUTPUT_DIR_IMAGES, exist_ok=True)

    for i, row in source_tweets.iterrows():
        tweet_id = row['tweetId']
        tweet_url = row['tweetUrl']
        print(f"Processing tweet ID: {tweet_id}")

        driver.get(tweet_url)
        time.sleep(15)

        new_row = {'tweetId': tweet_id, 'count': row['count'], 'tweetUrl': tweet_url}

        try:
            new_row['tweetDate'] = driver.find_element(By.XPATH, '//time').text
            new_row['username'] = driver.find_elements(By.CSS_SELECTOR, 'div a')[6].text
            full_text = driver.find_elements(By.CSS_SELECTOR, 'div div')[0].text
            new_row['text'] = driver.find_elements(By.CSS_SELECTOR, 'div span')[18].text
            images = driver.find_elements(By.XPATH, '//img[@alt="Image"]')
            if images:
                new_row['imageUrl'] = images[0].get_attribute('src')
                urllib.request.urlretrieve(new_row['imageUrl'], f"{OUTPUT_DIR_IMAGES}/image_{tweet_id}.jpg")
            else:
                new_row['imageUrl'] = None

            for key, pattern in patterns.items():
                match = pattern.search(full_text)
                new_row[key] = match.group(1) if match else None

        except Exception as e:
          print(f"An error occurred: {e}")
          for key in ['tweetDate', 'username', 'text', 'imageUrl', 'views', 'reposts', 'quotes', 'likes', 'bookmarks']:
             new_row[key] = "TWEET-NOT-FOUND"

        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df.to_csv(OUTPUT_FILE_INTERMEDIATE, index=False)

        if (i + 1) % 40 == 0:
            print("Pausing for 10 minutes after processing 40 tweets.")
            time.sleep(600)

    driver.quit()
    return df

In [ ]:
OUTPUT_DIR_IMAGES = "path-to-images-folder"
OUTPUT_FILE_INTERMEDIATE = "path-to-intermediate-file.csv"

source_tweets = community_notes['visual']['tweetId'].value_counts().reset_index()
source_tweets.columns = ['tweetId', 'count']

hydrated_df = collect_tweet_data(source_tweets,OUTPUT_DIR_IMAGES,OUTPUT_FILE_INTERMEDIATE)

### 3.3. Collect Data on Follower Counts

In [ ]:
def collect_followers_count(df,OUTPUT_FILE_USERS):
    driver = create_chrome_driver()
    df['userFollowers'] = None  # Initialize the userFollowers column

    for i, row in df.iterrows():
        username = row.get('username', None)

        if username is None or username == 'TWEET-NOT-FOUND':
            print("Skipping row with missing or TWEET-NOT-FOUND username.")
            df.at[i, 'userFollowers'] = 'TWEET-NOT-FOUND'
            continue

        username = username.lstrip('@')
        url = f"https://twitter.com/{username}"
        print(f"Collecting followers count for: {username}")

        driver.get(url)
        time.sleep(15)

        try:
            all_span_elements = driver.find_elements(By.CSS_SELECTOR, 'span span')
            for j, element in enumerate(all_span_elements):
                if element.text == 'Followers':
                    follower_count = all_span_elements[j - 1].text
                    df.at[i, 'userFollowers'] = follower_count
                    print(f"Follower count for {username}: {follower_count}")
                    break

        except Exception as e:
            print(f"An error occurred while collecting followers count for {username}: {e}")

        if (i + 1) % 40 == 0:
            print("Saving intermediate data.")
            df.to_csv(OUTPUT_FILE_USERS, index=False)
            print("Pausing for 10 minutes.")
            time.sleep(600)

    driver.quit()
    df.to_csv(OUTPUT_FILE_USERS, index=False)

In [ ]:
OUTPUT_FILE_USERS = "path-to-users-file.csv"

hydrated_df_users = collect_followers_count(hydrated_df)

### 3.4 Data Cleaning

Finally, to ensure the data's reliability and consistency for subsequent analyses, we clean the data prior to its use. This involves the standardization of numerical conventions and the transformation of the dates into a standard format (day/month/year).

In [ ]:
from datetime import datetime

def clean_dataframe(df):
    current_year = "2023"

    def transform_value(x):
        if pd.isna(x) or x == '': return '0'
        try:
            num = float(x.replace('M', '').replace('K', '').replace(',', ''))
            num *= 1e6 if 'M' in x else 1e3 if 'K' in x else 1
            return (f"{num/1e6:.1f}m" if num >= 1e6 else f"{num/1e3:.1f}k").rstrip('0').rstrip('.') if num >= 1e3 else str(int(num))
        except: return x

    def transform_date(d):
        if not isinstance(d, str): return None
        try: return datetime.strptime(d, '%I:%M %p · %b %d, %Y').strftime('%d/%m/%Y')
        except: pass
        try: return datetime.strptime(d, '%b %d, %Y').strftime('%d/%m/%Y')
        except: pass
        try: return datetime.strptime(f"{d}, {current_year}", '%b %d, %Y').strftime('%d/%m/%Y')
        except: return None

    for col in ['views', 'reposts', 'quotes', 'likes', 'bookmarks', 'userFollowers']:
        df[col] = df[col].astype(str).apply(transform_value)

    df['tweetDate'] = df['tweetDate'].apply(transform_date)
    df.rename(columns={'count': 'notesCount'}, inplace=True)

    return df

In [ ]:
OUTPUT_FILE_FINAL = "path-to-clean-file.csv"
processed_df = clean_dataframe(hydrated_df)
processed_df.to_csv(OUTPUT_FILE_FINAL, index=False)

# Codebook For Data Annotation

# **1. Introduction**

This codebook provides guidance on the manual coding process applies to the X dataframe obtained by searching tweets obtained from the Community Notes data releases. The dataset contains 16 columns, each capturing specific numeric and categorial aspects of a tweets.

| Column Name  | Description                                                 |
|--------------|-------------------------------------------------------------|
| tweetId      | Unique identifier for each tweet                            |
| tweetDate    | Date when the tweet was posted                   |
| username     | Username of the account that posted the tweet               |
| userFollowers| Number of followers of the user who posted the tweet        |
| verified     | Indicator if the user account is verified (TRUE/FALSE)      |
| noteCount    | Total count of community notes obtained by the tweet.       |
| tweetUrl     | URL directing to the tweet                                  |
| imageUrl     | URL of the accompanying image of the tweet                  |
| views        | Number of times the tweet has been viewed                   |
| reposts      | Number of times the tweet has been reposted                 |
| quotes       | Number of times the tweet has been quoted                   |
| likes        | Number of likes the tweet has received                      |
| bookmarks    | Number of times the tweet has been bookmarked               |
| political    | Label indicating if the tweet is POLITICAL or NON-POLITICAL |
| media        | Type of media attached to the tweet (IMAGE or VIDEO)        |
| text         | The text content of the tweet                               |



--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

# **2. Variables and Coding Instructions**


### **Status of Tweet**

**REMOVED**: If a tweet is no longer existing or accessible during the inspection, it is marked as 'REMOVED'.

**NO-DATA**: If a tweet was posted before the rollout of impression data in  and hence lacks impressions data, it is labeled as 'NO-DATA'. The data cutoff of impressions data can be quite inconsistent as this feature was rolled out progressively between November and December 2022

**NOT-AI-GEN**: If the media accompanying the tweet is not AI-generated, as determined by assessing community notes consensus, label the entry as 'NOT-AI-GEN'.

--------------------------------------------------------------------------------

### **User's Verified Status (Column Name: Verified)**

**TRUE**: The user posting the tweet has a verification tick. This includes any color of verification tick and also encompasses premium tiers such as organization verification.

**FALSE**: The user does not have a verification tick.



--------------------------------------------------------------------------------

### **Political Status (Column Name: Political)**

**POLITICAL**: Tweets that explicitly or implicitly referred to politicians, governments, political parties, or political issues


**NON-POLITICAL**: Tweets that do not meet the above criteria are labeled as 'NON-POLITICAL'.

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

# **3. Labelling Workflow**


**Initial Inspection**: Review each row to assess the status of the tweet. If the tweet has been removed or does not exist, label the status column as 'REMOVED'. If impression data is missing, label it as 'NO-DATA'.
AI Image Detection: Inspect the image accompanying each tweet to determine if it is AI-generated. If it is not, mark the status column as 'NOT-AI-GEN'. This last feature annotation will be based on community notes consensus noted during the inspection process.

**Verification Check**: Examine the profile of the user who posted the tweet and determine if they have a verification tick. Update the verified column accordingly with 'TRUE' or 'FALSE'.

**Political Nature**: Assess the content of the tweet to determine if it portrays a political figure. Update the POLITICAL column with either 'POLITICAL' or 'NON-POLITICAL' based on the evaluation.

--------------------------------------------------------------------------------
--------------------------------------------------------------------------------

# **4. Krippendorph's Alpha**


To ensure the consistency and reliability of the manual coding process, we employ **Krippendorff's Alpha** as a statistical measure of agreement among multiple raters. This  method allows us to evaluate the reliability across different levels of measurement, making it particularly suitable for a dataset with diverse variables like ours. Computing Krippendorff's Alpha helps validate the manual annotations, thereby adding additional rigor to our annotation process. <Add this later>